In [2]:
import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

import matplotlib 
matplotlib.use('Agg')

In [11]:
# global variables

char = "\\"
columns = ["date", "tavg", "tmin", "tmax", "prcp"]
t_columns = [i+"_x" for i in columns]
s_columns = [i+"_y" for i in columns]
op_into = False
empty_pred_target = "GqIUVenONyZikTIz"
save_plots = False

In [3]:
# get pathsfor daily and target

min_daily_weather_paths = [f for f in glob("../data/weather/minnesota_daily/*.csv")]
pred_targets_paths = [f for f in glob("../data/weather/prediction_targets_daily/*.csv") if empty_pred_target not in f]

print(len(min_daily_weather_paths))
print(len(pred_targets_paths))

97
197


In [4]:
# read in all daily weather csvs

min_daily_weather_dict = {}

for path in min_daily_weather_paths:

    station_code = path.split(char)[-1].split(".")[0]

    min_daily_weather_dict[station_code] = pd.read_csv(filepath_or_buffer=path, names=columns, index_col="date")
    
    if op_into:
        print(f"{station_code}:\n\tfrom: {min_daily_weather_dict[station_code].index[0]}\n\tto: {min_daily_weather_dict[station_code].index[-1]}")

In [5]:
# calculate weights for averaging features
# weight depends on nan values in each feature
# prediction targets are left out to prevent data leakage

nan_counts = np.zeros((len(min_daily_weather_paths), 4))

for i, path in enumerate(min_daily_weather_paths):
    num_nans = pd.read_csv(filepath_or_buffer=path, names=columns, index_col="date").isna().sum()
    nan_counts[i] = np.array(num_nans)

weights = np.sum(a=nan_counts, axis=0)
weights = 1 - (weights/np.sum(a=weights))

In [20]:
# read sequentally all prediction targets and compare with all station data

distances = {"Prediction Target": [],
             "Station": [],
             "L2 average distance": [],
             "L2 weighted distance": []}

for i, path in enumerate(pred_targets_paths):

    prediction_code = path.split(char)[-1].split(".")[0]

    df_target = pd.read_csv(filepath_or_buffer=path, names=columns, index_col="date")

    for station in min_daily_weather_dict.keys():
        
        # 1, check date correspondence
        tmp = pd.merge(left=df_target, right=min_daily_weather_dict[station], left_index=True, right_index=True, how="inner")

        if op_into:
            print(tmp)

        if tmp.empty:
            print(f"{prediction_code} and {station} have no matching dates.")

            distances["Prediction Target"].append(prediction_code)
            distances["Station"].append(station)
            distances["L2 average distance"].append(np.inf)
            distances["L2 weighted distance"].append(np.inf)

        else:
            # 2, calculate average L2 distance between all measured modalities
            dist_t_avg = (tmp["tavg_x"] - tmp["tavg_y"])**2
            dist_t_min = (tmp["tmin_x"] - tmp["tmin_y"])**2
            dist_t_max = (tmp["tmax_x"] - tmp["tmax_y"])**2
            dist_t_prcp = (tmp["prcp_x"] - tmp["prcp_y"])**2

            temp = np.nan_to_num(np.array([dist_t_avg, dist_t_min, dist_t_max, dist_t_prcp]), 0.0)

            avg_distance = np.nanmean(temp)
            weighted_distance = np.nanmean(weights*temp.T)

            distances["Prediction Target"].append(prediction_code)
            distances["Station"].append(station)
            distances["L2 average distance"].append(avg_distance)
            distances["L2 weighted distance"].append(weighted_distance)

df_distances = pd.DataFrame(data=distances)

GTpYgkFKgvhUVIdW and 9NRIJ have no matching dates.
GTpYgkFKgvhUVIdW and KD390 have no matching dates.
GTpYgkFKgvhUVIdW and KRYM0 have no matching dates.
GTpYgkFKgvhUVIdW and KY490 have no matching dates.
GTpYgkFKgvhUVIdW and KY630 have no matching dates.
GTpYgkFKgvhUVIdW and P6529 have no matching dates.
GTpYgkFKgvhUVIdW and UYB6K have no matching dates.
GTpYgkFKgvhUVIdW and X9FED have no matching dates.
GTpYgkFKgvhUVIdW and Z7ZOG have no matching dates.
IuUAGHGofIeJLHnH and 9NRIJ have no matching dates.
IuUAGHGofIeJLHnH and KD390 have no matching dates.
IuUAGHGofIeJLHnH and KY490 have no matching dates.
IuUAGHGofIeJLHnH and P6529 have no matching dates.
IuUAGHGofIeJLHnH and UYB6K have no matching dates.
IuUAGHGofIeJLHnH and X9FED have no matching dates.
IuUAGHGofIeJLHnH and Z7ZOG have no matching dates.
mMNhXEjWMwSvpLph and 9NRIJ have no matching dates.
mMNhXEjWMwSvpLph and P6529 have no matching dates.
mMNhXEjWMwSvpLph and UYB6K have no matching dates.
mMNhXEjWMwSvpLph and X9FED have

In [30]:
# Assuming you have a pandas DataFrame called df_distances
df_grouped = df_distances.groupby('Prediction Target')
df_sorted = df_grouped.apply(lambda x: x.sort_values('L2 weighted distance', ascending=True))

# Reset the index of the resulting DataFrame
df_result = df_sorted.reset_index(drop=True)
df_result.to_csv(path_or_buf="../data/intermedier/prediction_target_station_distances_L2.csv")

In [21]:
idx = df_distances.groupby('Prediction Target')['L2 weighted distance'].idxmin()
result = df_distances.loc[idx]
result

,Prediction Target,Station,L2 average distance,L2 weighted distance
78,ACAvNTuEuFWcmwms,KROS0,4.744662,3.896683
843,AWUpGSoLMOQdgmLf,KONA0,11.308500,9.141236
411,AmezwjsXKaggSICV,KCKN0,14.532802,13.048470
711,AvjrUfvxwfPdFfLD,KETH0,4.996847,2.832416
1091,BLEPTCvUWUfqXlBZ,KCNB0,10.479220,9.041338
...,...,...,...,...
18885,zNYjRZijEySKjImp,KONA0,8.577518,7.054493
18982,zThgIIrssGevRmsP,KONA0,19.038213,10.533649
18620,zhXchzMIBxaWiYPf,P6529,14.312543,8.441644
18799,zkNJkuOxjgWDUWIi,KROS0,2.338327,1.913806


In [15]:
result.sort_values("L2 weighted distance", ascending=True)

,Prediction Target,Station,L2 average distance,L2 weighted distance
3435,DfizCymEXgwIjnki,KFRM0,2.019714,1.643573
4000,DusycBGfBqLIQNFf,KCKN0,2.282793,1.800002
1039,bJfBssUbPvnDtmOe,KOTG0,2.680624,1.862679
18799,zkNJkuOxjgWDUWIi,KROS0,2.338327,1.913806
8708,KVYsyLIvhwwqgSun,KPQN0,2.561615,2.052540
...,...,...,...,...
5410,GTpYgkFKgvhUVIdW,KPQN0,21.535062,20.257330
15710,VasKmVkDnzxJzhep,P6529,28.474237,20.443796
12958,RfnwXzFAVgaCkooQ,KLYV0,25.811087,22.553979
18236,YRbFthxdZAHOIvEq,72644,26.685666,26.254729


In [9]:
# load station - county mapper

station_county_map = pd.read_csv(filepath_or_buffer="../data/intermedier/stations_county_affiliation.csv")

In [10]:
# save results in csv

final = pd.merge(left=result, right=station_county_map, left_on="Station", right_on="Station")
final[["Prediction Target", "Station", "County"]].to_csv(path_or_buf="../data/intermedier/prediction_target_station_matching.csv", index=False)

In [12]:
# plot closest parts

color_s = "#0B2447"
color_t = "#A5D7E8"

if save_plots:

    for pair in result.iterrows():

        station = pair[1]["Station"]
        target = pair[1]["Prediction Target"]

        statioin_path = [s for s in min_daily_weather_paths if station in s][0]
        target_path = [s for s in pred_targets_paths if target in s][0]

        df_station = pd.read_csv(filepath_or_buffer=statioin_path, names=columns, index_col="date")
        df_target = pd.read_csv(filepath_or_buffer=target_path, names=columns, index_col="date")

        tmp = pd.merge(left=df_station, right=df_target, left_index=True, right_index=True, how="inner", suffixes=["_station", "_target"])

        corr = tmp.corr()

        with plt.ioff():
        
            title = f"{station} - {target}: {station_county_map[station_county_map['Station'] == station]['County'].values[0]}"

            fig, ax = plt.subplots(ncols=1, nrows=4, figsize=(18, 8))

            ax[0].set_title(title)

            ax[0].plot(tmp.index, tmp["tavg_station"], label="tavg_station", linewidth=2, alpha=0.75, color=color_s)
            ax[0].plot(tmp.index, tmp["tavg_target"] , label="tavg_target", linewidth=2, alpha=0.75, color=color_t)
            ax[0].text(0, 0, f"corr: {corr['tavg_station']['tavg_target']}", fontsize=12)
            ax[0].set_xticks(tmp.index[::int(len(tmp.index)/10)])
            ax[0].set_ylabel("°C")
            ax[0].legend()

            ax[1].plot(tmp.index, tmp["tmin_station"], label="tmin_station", linewidth=2, alpha=0.75, color=color_s)
            ax[1].plot(tmp.index, tmp["tmin_target"] , label="tmin_target", linewidth=2, alpha=0.75, color=color_t)
            ax[1].text(0, 0, f"corr: {corr['tmin_station']['tmin_target']}", fontsize=12)
            ax[1].set_xticks(tmp.index[::int(len(tmp.index)/10)])
            ax[1].set_ylabel("°C")
            ax[1].legend()

            ax[2].plot(tmp.index, tmp["tmax_station"], label="tmax_station", linewidth=2, alpha=0.75, color=color_s)
            ax[2].plot(tmp.index, tmp["tmax_target"] , label="tmax_target", linewidth=2, alpha=0.75, color=color_t)
            ax[2].text(0, 0, f"corr: {corr['tmax_target']['tmax_target']}", fontsize=12)
            ax[2].set_xticks(tmp.index[::int(len(tmp.index)/10)])
            ax[2].set_ylabel("°C")
            ax[2].legend()

            ax[3].plot(tmp.index, tmp["prcp_station"], label="prcp_station", linewidth=2, alpha=0.75, color=color_s)
            ax[3].plot(tmp.index, tmp["prcp_target"] , label="prcp_target", linewidth=2, alpha=0.75, color=color_t)
            ax[3].text(0, 0, f"corr: {corr['prcp_target']['prcp_target']}", fontsize=12)
            ax[3].set_xticks(tmp.index[::int(len(tmp.index)/10)])
            ax[3].set_ylabel("mm")
            ax[3].legend()

            plt.savefig(fname=f"../data/intermedier/distance/{station}_{target}.png",
                        bbox_inches="tight",
                        pad_inches = 0.1,
                        dpi=100)
            
            plt.close()
        